# European Dynamic Energy Prices Analysis



<a id="0"></a>
# Table of Contents

1. [Tech Stack](#ts)
2. [Goals](#goal)
3. [Architecture](#arch)
4. [Scraping Data](#sd)
5. [Storing in Delta Tables](#delta)
6. [Scheduling](#schd)
7. [Transformations in DBT](#dbt)
8. [Power BI Report](#pbi)

<a id="ts"></a>
# Tech Stack


|Use Case|Local| Fabric Equivalent | 
|:------:| :---------: |  :-------: | 
|Programming Language|python|python|
|Web Scraping|Beautiful Soup 4| Beautiful Soup 4 |
|Data Storage| DuckDB\| Delta Tables| OneLake\| Lakehouse\| Delta Tables|
|Data Transformations|dbt|dbt |
|Visualization|Power BI Desktop| Power BI Desktop/|Power BI Service |
|Scheduling| Airflow | Pipeline|


<div class="alert alert-block alert-info" style="color:black">
Other Fabric Components that would have been used:<br>
1. <b>Notebook:</b> for running PySpark jobs<br>
2. <b>Lakehouse SQL endpoint:</b> for storing results from dbt runs. </div>

[Back to Top](#0)

<a id="goal"></a>
# Goals

<div class="alert alert-block alert-info" style="color:black">
    The origial idea was to build a project on <b>microsoft Fabric</b> to apply its capabilites for a typical end to end project.


But currently Microsoft Fabric has Issues and it would not allow me purchase/trial any capacity no matter what options I tried.

</div>
<div class="alert alert-block alert-warning" style="color:black">
<a href="https://www.skool.com/microsoft-fabric/problems-starting-a-new-fabric-free-trials-an-update" style="color:blue; text-decoration:underline"">ISSUE DETAILS HERE</a>
</br>
So I decided to build this project locally for now and maybe apply Fabric features in the future once the issues are sorted out.
</div>

***

Since European markets moved to **dynamic energy prices** which brings challenges, but also lots of opportunities. By analyzing the prices, one can optimize your energy consumption and save money. The flexibility and options improve a lot with the installation of a home battery. 

In this porject will be scraping Epex Spot (European Energy Exchange) day-ahead energy pricing data. Energy companies buy and sell energy on this exchange. The price of energy is announced one day in advance. The price can even become negative when there will be too much energy on the grid.

***

<div class="alert alert-block alert-info" style="color:black">
The goals of this project are as follows:
</br>
1. Web Scrape data from <a href="https://www.epexspot.com/" style="color:blue; text-decoration:underline">Epex Spot</a> (European Energy Exchange) day-ahead energy pricing data. Energy companies buy and sell energy on this exchange. The price of energy is announced one day in advance. The price can even become negative when there will be too much energy on the grid (e.g. it’s sunnier and windier than expected and some energy plants cannot easily scale down capacity)
</br>
Unfortunately, Epec Spot website does not offer a free API to get the data, so I will use <b>BeautifulSoup4</b> to scrape this data.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<img src='./images/fabric_project/epexspot.png' stlye='display: block; margin: 0 auto' alt="Epex Spot"  width="700" height="600" >
</br>
2. Store it in <b>Delta Tables</b> locally

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;At its very core, a <b>Delta Tables</b> is a bunch of Parquet files stored in a data lake, with a special transaction log that keeps track of all the changes made to the data. This log is the secret sauce that makes Delta Tables so powerful. It enables cool features like ACID transactions, data versioning, schema enforcement, and performance optimizations.

<div class="alert alert-block alert-warning" style="color:black">
    I will use <b>Duck DB</b> as the local database for reading and writing to local <b>Delta Tables</b>
</div>
<a href="https://www.chaosgenius.io/blog/delta-table/" style="color:blue; text-decoration:underline">SOURCE</a>
</br>
3. Use <b>dbt</b> for transforming data for creating a report in Power BI

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;dbt is a tool for analytics engineers where you can use SQL with Jinja templating to make that SQL easier to use and add functionalities. By using dbt, we can leverage a lot of best practices from software engineering in our analytics projects.
</br>
4. Use tranformed data in <b>Power BI</b> for creating a Report
</div>


[Back to Top](#0)

<a id='arch'></a>
# Architecture

***

If it were the project in **Microsoft Fabric**, then below was the Architecture I planned

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<img src='./images/fabric_project/architecture-1.png' stlye='display: block; margin: 0 auto' alt="Micrsoft Fabric Project Architecture"  width="700" height="600" >


[Back to Top](#0)

<a id="sd"></a>
# Scraping Data
***
###  1. Beautifulsoup4 Installion

##### Local Environment
***
<div class="alert alert-block alert-info" style="color:black">

<ol>
    <li>Install Python. Since I planned to use PySpark, I installed version 3.10 as that is the one seems to be working on my machine. </li>
    <li>Create a Virtual Env</li>
    <li>
        Install Beautifulsoup4
        <br>
        <code>pip install beautifulsoup4</code>
    </li>
   
</ol>
</div>

##### Steps I would have followed in Fabric
***

<div class="alert alert-block alert-info" style="color:black">
    Since I would use <b>beautifulsoup4</b> Python package in the Spark runtime environment of our Fabric Workspace, I needed to install it first. 
    <ol>
        <li>This can be done by going to Fabric Workspace settings and then expanding the Data Engineering/Science section. </li>
    <li>Click on </> Library management tab. </li>
    <li>Click on the + Add from PyPI button. In the text box that appears, enter <b>beautifulsoup4</b>, pick the latest version, and click on the Apply button. This will install the package in the Workspace.</li>

<br>
<a href="https://fabric.guru/installing-custom-python-packages-in-fabric" style="color:blue; text-decoration:underline">
        How to Install a Python package Microsoft Fabric
</a>
</br>
The steps can be seen in the image below
</div>



&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<img src='./images/fabric_project/install_lib_python_fabric.png' stlye='display: block; margin: 0 auto' alt="Micrsoft Fabric Install Python Library"  width="700" height="600" >


### 2. PySpark Installation
***

##### Local Environment
<div class="alert alert-block alert-info" style="color:black">
 Please follow steps from my other project on PySpark Installation 
    <a href="https://gsinghmath.pythonanywhere.com/static/Uber%20Data%20%20Analysis%20PySpark.html" style="color:blue; text-decoration:underline">here</a>

I also needed to install below python packages.


<ol>
        <li>pandas</li>
        <li>requests</li>
        <li>delta-spark</li>
        <li>pyspark</li>
        <li>jupyterlab</li>
</ol>

    
</div>
    
##### Fabric
***
<div class="alert alert-block alert-info" style="color:black">
I would have created a new <b>Notebook</b> and followed Fabric's prompts to also create a new Lakehouse. Notebook's allow offer a PySpark enviroment.
<br>
<a href="https://www.deeplearningnerds.com/microsoft-fabric-how-to-create-a-notebook/" style="color:blue; text-decoration:underline">
        How to Create a New Notebook Microsoft Fabric
</a>

</br>
Above steps can be seen in Image below
</div>


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<img src='./images/fabric_project/notebook_fabric.png' stlye='display: block; margin: 0 auto' alt="Micrsoft Fabric New Notebook"  width="700" height="600" >



### 3. Web Scraping Code
***

##### Local Environment
***

<div class="alert alert-block alert-info" style="color:black">
The code in the next Cell i
    s summarized as below:

<ol>
    <li>I created a <b>python class: EpexSpotEnergyPrices</b> to scrap data from Epex Spot website.</li>
    <li>I submit an <b>HTTP POST request</b> to the web page which lists the day-ahead data for a chosen market in a table and the server responds with an <b>HTML</b> page. I also pass necessary parameters as expected by the webpage.</li>
    <li>Parse the <b>HTML</b> page using <b>Beautifulsoup4</b> and extract the required table.
    <li>I also convert the date and time to UTC, so that I don't have to worry about timezones or daylight-saving time later on. The results are stored in a <b>Pandas DataFrame</b></li>
    <li>Then in the next cell I instantiate the class and extract the data in <b>Pandas DataFrame</b></li>
    <li>In next cell, I use create a <b>spark session</b> and save the extracted data to <b>Delta Format and partition it by market </b> in table <b>epex_spot_prices</b></li>
    <li>I also make sure the next run, the data is appended to the same table, since I wish to <b>schedule</b> in <b>Airflow</b>  for daily data extraction.</li>
    <li>The resulting data with <b>market paritioning</b> would look like below</li>
</ol>
</div>


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<img src='./images/fabric_project/scraping_result.png' stlye='display: block; margin: 0 auto' alt="Local Delta Table with Market Paritioning"  width="300" height="400" >


##### Fabric
***
<div class="alert alert-block alert-info" style="color:black">
I would have created a function that received <b>market</b> as a parameter and scraped data for that <b>market</b> from <b>Epex Spot</b> Website. Basically modifiying the code as explained above. The result would have been saved in <b>Delta Table</b> in the <b>Lakehouse</b>.
</div>


[Back to Top](#0)

In [1]:
from datetime import date, datetime, timedelta, timezone
from typing import Any
import pandas as pd
import requests
from bs4 import BeautifulSoup
import warnings
# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

class EpexSpotEnergyPrices:
    """Scrape data from Epex Spot website as they do not offer any API.
       They provide Energy Auction data a day ahead, so its only possible to get data from today onwards.
       url: https://www.epexspot.com/en/market-results
       This has been updated recently.
       we get the data for these markets: ['AT','BE','CH','DE-LU','DK1','DK2','FI','FR','GB','NL','NO1','NO2',
                                           'NO3','NO4','NO5','PL','SE1','SE2','SE3','SE4']
        Usage:
        ep = EpexSpotEnergyPrices()
        ep.run()
    """
    def __init__(self):
        self.name = 'learn'
    def _to_float(self, v: str) -> float:
        return float(v.replace(",", ""))
    
    def _as_date_str(self, v: date) -> str:
        return v.strftime("%Y-%m-%d")
    
    
    def fetch_data(self, delivery_date: date, market_area: str) -> dict[str, Any]:
        trading_date = delivery_date - timedelta(days=1)
        params = {
            "market_area": market_area,
            "trading_date": self._as_date_str(trading_date),
            "delivery_date": self._as_date_str(delivery_date),
            'auction': 'MRC',
            "modality": "Auction",
            "sub_modality": "DayAhead",
            "product": "60",
            "data_mode": "table",
        }
        data = {
            "form_id": "market_data_filters_form",
            "_triggering_element_name": "submit_js",
        }
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:127.0) Gecko/20100101 Firefox/127.0'}
        r = requests.post("https://www.epexspot.com/en/market-results", params=params, data=data, headers=headers)
        r.raise_for_status()
        return str(r.content)
    
    def extract_table_data(self, delivery_date: datetime, data: dict[str, Any], market_area: str) -> pd.DataFrame:
        soup = BeautifulSoup(data, features="html.parser")
    
        try:
            table = soup.find("table", class_="table-01 table-length-1")
            body = table.tbody
            rows = body.find_all_next("tr")
        except AttributeError:
            return pd.DataFrame.from_records([], columns=["market", "start_time", "end_time", "buy_volume", "sell_volume", "volume", "price"])
     # no data available
    
        start_time = delivery_date.replace(hour=0, minute=0, second=0, microsecond=0)
    
        # convert timezone to UTC (and adjust timestamp)
        start_time = start_time.astimezone(timezone.utc)
    
        records = []
        for row in rows:
            end_time = start_time + timedelta(hours=1)
            buy_volume_col = row.td
            sell_volume_col = buy_volume_col.find_next_sibling("td")
            volume_col = sell_volume_col.find_next_sibling("td")
            price_col = volume_col.find_next_sibling("td")
            records.append(
                (
                    market_area,
                    start_time,
                    end_time,
                    self._to_float(buy_volume_col.string),
                    self._to_float(sell_volume_col.string),
                    self._to_float(volume_col.string),
                    self._to_float(price_col.string),
                )
            )
            start_time = end_time
        return pd.DataFrame.from_records(records, columns=["market", "start_time", "end_time", "buy_volume", "sell_volume", "volume", "price"])
    
    def fetch_day(self, delivery_date: datetime, market_area) -> pd.DataFrame:
        data = self.fetch_data(delivery_date.date(), market_area)
        return self.extract_table_data(delivery_date, data, market_area)
    def run(self):
        result_df = pd.DataFrame(columns=['market', 'start_time', 'end_time', 'buy_volume', 'sell_volume',
       'volume', 'price'])
        markets = ['AT','BE','CH','DE-LU','DK1','DK2','FI','FR','GB','NL','NO1','NO2',
                                           'NO3','NO4','NO5','PL','SE1','SE2','SE3','SE4']
        for market in markets:
            delivery_date = datetime.now() + timedelta(days=1)
            print("Extracting data for", market, 'for ', delivery_date)
            prices = self.fetch_day(delivery_date, market_area=market)
            result_df = pd.concat([result_df, prices], ignore_index=True)
            
        return result_df

In [2]:
ep = EpexSpotEnergyPrices()
data =  ep.run()

Extracting data for AT for  2025-04-19 11:52:59.591890
Extracting data for BE for  2025-04-19 11:53:00.752151
Extracting data for CH for  2025-04-19 11:53:01.667114
Extracting data for DE-LU for  2025-04-19 11:53:02.620590
Extracting data for DK1 for  2025-04-19 11:53:03.835446
Extracting data for DK2 for  2025-04-19 11:53:04.536127
Extracting data for FI for  2025-04-19 11:53:05.471504
Extracting data for FR for  2025-04-19 11:53:06.219133
Extracting data for GB for  2025-04-19 11:53:06.945077
Extracting data for NL for  2025-04-19 11:53:07.688589
Extracting data for NO1 for  2025-04-19 11:53:08.448961
Extracting data for NO2 for  2025-04-19 11:53:09.415484
Extracting data for NO3 for  2025-04-19 11:53:10.148543
Extracting data for NO4 for  2025-04-19 11:53:11.378019
Extracting data for NO5 for  2025-04-19 11:53:12.159536
Extracting data for PL for  2025-04-19 11:53:12.942962
Extracting data for SE1 for  2025-04-19 11:53:13.882953
Extracting data for SE2 for  2025-04-19 11:53:14.65232

<a id="delta"></a>
# Storing in Delta Tables

[Back to Top](#0)

In [3]:
from delta.tables import *
from pyspark.sql.functions import *
import pyspark
from delta import *
import os
import sys
from  pyspark.errors import AnalysisException

def create_spark_session():
    os.environ['PYSPARK_PYTHON'] = sys.executable
    os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
    os.environ['HADOOP_HOME'] = os.getcwd() + "\\winutils-master\\hadoop-3.3.5\\"
    os.environ['JAVA_HOME'] = "C:\\Users\\gmatharu\\Downloads\\microsoft_fabric_proj\\jdk-8u441-windows-x64\\jdk1.8.0_441\\"
    os.environ['path'] = os.environ['path'] +';'+ os.getcwd() + '\\winutils-master\\hadoop-3.3.5\\bin\\'
    os.environ['SPARK_HOME'] = "C:\\Users\\gmatharu\\Downloads\\microsoft_fabric_proj\\spark-3.5.5-bin-hadoop3\\"
    
    builder = pyspark.sql.SparkSession.builder.appName("EpexSpotEnergyPrices")\
              .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
              .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    try:
        spark = configure_spark_with_delta_pip(builder).getOrCreate()
    except:
        print("Cannot Create Spark Session, please check your spark installation.Exiting...")
        sys.exit()
    return spark
def save_delta_table(data):
    spark = create_spark_session()
    spark_df = spark.createDataFrame(data)
    try:
        current = spark.read.format("delta").load('epex_spot_prices')
        current = current.union(spark_df)
        current.write.mode('overwrite').format("delta").save("epex_spot_prices")
        print((current.count(), len(current.columns)), (spark_df.count(), len(spark_df.columns)))
    except AnalysisException:
        print("No previous data exist")
        spark_df.write.format("delta").option("overwriteSchema", "true").partitionBy("market").save("epex_spot_prices")

save_delta_table(data)
#.mode('overwrite').option("overwriteSchema", "true")

No previous data exist


<a id="schd"></a>
# Scheduling
***
##### Local
***
<div class="alert alert-block alert-info" style="color:black">
In the next iteration of this project I would use <b>Apache Airlfow</b> for creating mutliple <b>DAGS</b> and scheduling and creating a proper<b>Data Pipeline</b>, but to keep this project manageable, I scheduled the script to run everyday at <b>12:00PM</b> in <b>Window’s Task Scheduler</b>. If you linux feel free to use <b>Cron</b>
</div>

##### Fabric
***
<div class="alert alert-block alert-info" style="color:black">
I would have created a <b>paramteric Pipeline</b> in <b>Fabric</b> as the code I would have written to extract data would be for a single market. Also, <b>Fabric Pipelines</b> are just <b>Apache Airflow DAGS</b> underneath using a visual designer. So my local work would be parity matched to Fabric.


I would have followed below steps:
    <ol>
        <li>Create a Pipeline</li>
        <li>Use the <b>ForEach Activity</b>, since I want to repeat the process for every market
        Clicking on the <b>ForEach Activity</b> and then on the Settings tab would allow me to specify the collection I want to loop over. In our case, we want to loop over the markets, so we can use the following expression to build an array of the markets as strings:

<code> @createArray('AT','BE','CH','DE-LU','DK1','DK2','FI','FR','GB','NL','NO1','NO2','NO3','NO4','NO5','PL','SE1','SE2','SE3','SE4') </code>

To enter the expression,I would open the Pipeline expression builder by clicking on the Add dynamic content link that appears after highlighting the empty text box after Items. The expression builder will open and I would enter the above expression.
<br>

</li>
<li>
    Now that I have a loop that goes over the markets, I needed to tell the Pipeline what it should do on every iteration of the loop. Inside the <b>ForEach Activity</b> block, clicking the ✏️ icon would change the canvas to the Activities that are inside the <b>ForEach Activity</b>. 
</br>
Inside this canvas, I would add an Activity to run a Notebook by clicking on the <b>Notebook Activity</b> at the top. Basically, the <b>ForEach Activity</b> would run the <b>Notebook Activity</b> for each market.
</li>
<li>
    In the settings of the <b>Notebook Activity</b> select the Notebook from the dropdown and then expand the <b>Base parameters</b> section and use the <b>+ New button</b> to add a new parameter to the list. <b>The name of the parameter should match the name of the variable in the Notebook</b>.I would set the <b>Type to String</b>. 
    
</br>Now I have to link the parameter value to the loop variable from the <b>ForEach Activity</b> since I would be looping over every market. To do this, I would open the expression builder for this activity and use the following expression:

<code>@item()</code>
</li>
<li>After <b>Saving my work & Validating the pipelinw</b>, I would test it by <b>running it.</b></li>
<li>I would Schedule the Pipeline by clicking the <b>Schedule icon</b></li>
</ol>

<a href="https://databear.com/understanding_data_pipelines_microsoft_fabric/" style="color:blue; text-decoration:underline">
REFERENCE 1
</a>
</br>
<a href="https://www.red-gate.com/simple-talk/databases/sql-server/bi-sql-server/how-to-build-metadata-driven-pipelines-in-microsoft-fabric/" style="color:blue; text-decoration:underline">
REFERENCE 2
</a>
</div>


[Back to Top](#0)

<a id="dbt"></a>
# Transformations in DBT
***

<div class="alert alert-block alert-warning" style="color:black">
    <b>Note:</b> I am not focussed on modelling for this project.
</div>


<div class="alert alert-block alert-info" style="color:black">
I created a dbt project named <code>eur_enegry_prices</code>
</br>
<ol>
    <li>I would use DBT locally in both local and Fabric environment to creaate data tranformations. I would have also used <b>DataFlow Gen2</b>, but Since I don't have access to Fabric I would prefer DBT.</li>
    <li>Setup dbt, get the SQL endpoint from Fabric. Locally, since I would use DuckDB I set up my dbt profile as below
</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<img src='./images/fabric_project/dbt_profile.png' stlye='display: block; margin: 0 auto' alt="Local DBT profile"  width="250" height="300" >
</li>
    <li>Check all tranformations I build in my github repo for this project,
    <a href="https://github.com/gsinghmath" style="color:blue; text-decoration:underline">here</a>
    </br>One of them is <b>Simple Advice</b> suggesting to use electricity intelligently by using home battery by charging it when the price is low and discharging it when the price is high. Will be clear from Power BI Report.
</li>
    <li>Here are the screenshots of the <b>lineage and the macros and models</b> I built taken from dbt docs website</li>

</ol>
</div>

<img src='./images/fabric_project/dbt_project.png' stlye='display: block; margin: 0 auto' alt="Local DBT project"  width="800" height="600"><br>
<img src='./images/fabric_project/dbt_database.png' stlye='display: block; margin: 0 auto' alt="Local DBT database"  width="800" height="600" ><br>
<img src='./images/fabric_project/dbt_lineage.png' stlye='display: block; margin: 0 auto' alt="Local DBT lineage"  width="800" height="600" >


[Back to Top](#0)

<a id="pbi"></a>
# Power BI Report
***
<div class="alert alert-block alert-info" style="color:black">
    In order to connect Power BI to DuckDB I needed a Custom Connector. I used the connector provided 
    <a href="https://motherduck.com/docs/integrations/bi-tools/powerbi/" style="color:blue; text-decoration:underline">here</a>.
</div>

<div class="alert alert-block alert-warning" style="color:black">
    <b>Note:</b> 
    </br>I am not focussed on creating a proper dashboard, its just some basic report to showcase an end to end project. Also I cannot use Map Chart as Power BI desktop won't show them for me.
</div>

<div class="alert alert-block alert-info" style="color:black">
I built four reports showing various stats
<ol>
    <li><b>Average Prices:</b> showing average <b>Monthly</b> and <b>Daily</b> prices across Countries and Markets.   
        </br>
        <img src='./images/fabric_project/pbi_avp_prices.png' stlye='display: block; margin: 0 auto' alt="Power BI Avg Prices"  width="800" height="600" >
    </li>
    <li><b>Price Points and Suggestions Market:</b> Price Stats and Daily suggestions for a chosen market every hour based on price
        </br>   
        <img src='./images/fabric_project/pbi_suggestion.png' stlye='display: block; margin: 0 auto' alt="Power BI Suggestion"  width="800" height="600" >    
    </li>
    <li><b>Daily Price Comparison across Markets:</b> Price Comparison across markets for last 7 days
        </br>    
        <img src='./images/fabric_project/pbi_7days.png' stlye='display: block; margin: 0 auto' alt="Power BI 7 days"  width="800" height="600" > 
    </li>
    <li><b>Price Range Markets:</b> Shows the min vs max price for a chosen market
        </br>
        <img src='./images/fabric_project/pbi_price_range.png' stlye='display: block; margin: 0 auto' alt="Power BI Price Range"  width="800" height="600" >
    </li>
</ol>
</div>


[Back to Top](#0)